In [23]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(stringr)
    library(tidyr)
    library(tibble)
})

In [ ]:
# RNA

## Load data
obj1 <- readRDS("data/PBMC.single.cell.RDS")
counts <- GetAssayData(obj1)
metadata <- obj1@meta.data
obj1 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj1@meta.data$Tissue <- 'PBMC'
obj1@meta.data$Donor <- obj1@meta.data$Sample
obj1@meta.data$THERAPY <- obj1@meta.data$Therapy
obj1$Type <- gsub("HC", "Healthy Control", obj1$Type)
obj1$Type <- gsub("CC_NI", "CC Non-Inflamed", obj1$Type)
obj1$Type <- gsub("CC_I", "CC Inflamed", obj1$Type)
obj1@meta.data$Stim <- obj1@meta.data$Type
obj1@meta.data$Cell_Type <- obj1@meta.data$Cluster
obj1@meta.data <- obj1@meta.data[, !(colnames(obj1@meta.data) %in%  c('cdr3s_nt' ,'Sample', 'Therapy', 'Pool', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj1 <- SCTransform(obj1, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj1)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj1@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj1)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj1@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj1
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)
save(seu, file = file.path("objects", "PBMC.rda"))

scConf1 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf1, gene.mapping = TRUE, shiny.prefix = "sc1", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "SCT")

# ADT

## Load data
obj1 <- readRDS("data/PBMC.single.cell.RDS")
counts <- GetAssayData(obj1, assay = "ADT")
metadata <- obj1@meta.data
obj1 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj1@meta.data$Tissue <- 'PBMC'
obj1@meta.data$Donor <- obj1@meta.data$Sample
obj1@meta.data$THERAPY <- obj1@meta.data$Therapy
obj1$Type <- gsub("HC", "Healthy Control", obj1$Type)
obj1$Type <- gsub("CC_NI", "CC Non-Inflamed", obj1$Type)
obj1$Type <- gsub("CC_I", "CC Inflamed", obj1$Type)
obj1@meta.data$Stim <- obj1@meta.data$Type
obj1@meta.data$Cell_Type <- obj1@meta.data$Cluster
obj1@meta.data <- obj1@meta.data[, !(colnames(obj1@meta.data) %in%  c('cdr3s_nt' ,'Sample', 'Therapy', 'Pool', 'Cluster', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj1 <- SCTransform(obj1, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj1)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj1@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj1)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj1@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj1
# DefaultAssay(seu) <- 'ADT'
# VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu <- NormalizeData(seu, normalization.method = 'CLR', margin = 2)
seu <- ScaleData(seu)
seu <- RunPCA(seu, reduction.name = 'apca', verbose = TRUE)
seu <- RunUMAP(seu, reduction = "apca", reduction.name= "aumap",  dims = 1:2)

scConf2 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf2, gene.mapping = TRUE, shiny.prefix = "sc2", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "ADT")

In [ ]:
# RNA

## Load data
obj2 <- readRDS("data/CD3.single.cell.RDS")
counts <- GetAssayData(obj2)
metadata <- obj2@meta.data
obj2 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj2@meta.data$Tissue <- 'Colon'
obj2@meta.data$Donor <- obj2@meta.data$Sample
obj2@meta.data$Stim <- obj2@meta.data$Type
obj2@meta.data$Cell_Type <- obj2@meta.data$Cluster
obj2@meta.data <- obj2@meta.data[, !(colnames(obj2@meta.data) %in%  c('Dataset', 'Reaction', 'CDR3_NT', 'Cluster'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj2 <- SCTransform(obj2, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj2)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj2@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj2)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj2@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj2
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)
save(seu, file = file.path("objects", "CD3.rda"))

scConf3 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf3, gene.mapping = TRUE, shiny.prefix = "sc3", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "SCT")

# ADT

## Load data
obj2 <- readRDS("data/CD3.single.cell.RDS")
# counts <- GetAssayData(obj2, assay = "ADT")
# metadata <- obj2@meta.data
# obj2 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj2@meta.data$Tissue <- 'Colon'
obj2@meta.data$Donor <- obj2@meta.data$Sample
obj2@meta.data$Stim <- obj2@meta.data$Type
obj2@meta.data$Cell_Type <- obj2@meta.data$Cluster
obj2@meta.data <- obj2@meta.data[, !(colnames(obj2@meta.data) %in%  c('Dataset', 'Reaction', 'CDR3_NT', 'Cluster'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj2 <- SCTransform(obj2, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj2)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj2@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj2)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj2@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj2
DefaultAssay(seu) <- 'ADT'
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu <- NormalizeData(seu, normalization.method = 'CLR', margin = 2)
seu <- ScaleData(seu)
seu <- RunPCA(seu, verbose = TRUE)
seu@reductions[["umap"]] <- NULL

scConf4 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf4, gene.mapping = TRUE, shiny.prefix = "sc4", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "ADT")

In [ ]:
# RNA

## Load data
obj3 <- readRDS("data/epithelium.single.cell.RDS")
counts <- GetAssayData(obj3)
metadata <- obj3@meta.data
obj3 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj3@meta.data$Tissue <- 'Epithelium'
obj3@meta.data$Donor <- obj3@meta.data$Sample
obj3@meta.data$Stim <- obj3@meta.data$Type
obj3@meta.data$Cell_Type <- obj3@meta.data$Cluster
obj3@meta.data <- obj3@meta.data[, !(colnames(obj3@meta.data) %in%  c('Sample', 'Pool', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj3 <- SCTransform(obj3, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj3)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj3@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj3)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj3@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj3
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)
save(seu, file = file.path("objects", "epithelium.rda"))

scConf5 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf5, gene.mapping = TRUE, shiny.prefix = "sc5", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "SCT")

# ADT

## Load data
obj3 <- readRDS("data/epithelium.single.cell.RDS")
# counts <- GetAssayData(obj3, assay = "ADT")
# metadata <- obj3@meta.data
# obj3 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj3@meta.data$Tissue <- 'Epithelium'
obj3@meta.data$Donor <- obj3@meta.data$Sample
obj3@meta.data$Stim <- obj3@meta.data$Type
obj3@meta.data$Cell_Type <- obj3@meta.data$Cluster
obj3@meta.data <- obj3@meta.data[, !(colnames(obj3@meta.data) %in%  c('Sample', 'Pool', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj3 <- SCTransform(obj3, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj3)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj3@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj3)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj3@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj3
DefaultAssay(seu) <- 'ADT'
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu <- NormalizeData(seu, normalization.method = 'CLR', margin = 2)
seu <- ScaleData(seu)
seu <- RunPCA(seu, verbose = TRUE)
seu@reductions[["umap"]] <- NULL

scConf6 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf6, gene.mapping = TRUE, shiny.prefix = "sc6", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "ADT")

In [9]:
# RNA

## Load data
obj4 <- readRDS("data/stroma.and.immune.single.cell.RDS")
counts <- GetAssayData(obj4)
metadata <- obj4@meta.data
obj4 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj4@meta.data$Tissue <- 'Stromal_Immune'
obj4@meta.data$Donor <- obj4@meta.data$Sample
obj4@meta.data$Stim <- obj4@meta.data$Type
obj4@meta.data$Cell_Type <- obj4@meta.data$Cluster
obj4@meta.data <- obj4@meta.data[, !(colnames(obj4@meta.data) %in%  c('Sample', 'cdr3s_nt', 'Pool', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj4 <- SCTransform(obj4, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj4)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj4@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj4)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj4@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj4
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)
save(seu, file = file.path("objects", "stroma.rda"))

scConf7 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf7, gene.mapping = TRUE, shiny.prefix = "sc7", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "SCT")

# ADT

## Load data
obj4 <- readRDS("data/stroma.and.immune.single.cell.RDS")
counts <- GetAssayData(obj4, assay = "ADT")
metadata <- obj4@meta.data
obj4 <- CreateSeuratObject(counts = counts, meta.data = metadata)

## Clean columns
obj4@meta.data$Tissue <- 'Stromal_Immune'
obj4@meta.data$Donor <- obj4@meta.data$Sample
obj4@meta.data$Stim <- obj4@meta.data$Type
obj4@meta.data$Cell_Type <- obj4@meta.data$Cluster
obj4@meta.data <- obj4@meta.data[, !(colnames(obj4@meta.data) %in%  c('Sample', 'cdr3s_nt', 'Pool', 'Cluster', 'Type'))]

## Normalize
options(future.globals.maxSize = 4000 * 1024^2)
obj4 <- SCTransform(obj4, verbose = TRUE)

## Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()
### Blueprint annnotation
sce <- LayerData(obj4)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj4@meta.data$blueprint.main <- blueprint.main$pruned.labels
### Monaco annontation
sce <- LayerData(obj4)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj4@meta.data$monaco.fine <- monaco.fine$pruned.labels

## Cluster
seu <- obj4
DefaultAssay(seu) <- 'ADT'
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu <- NormalizeData(seu, normalization.method = 'CLR', margin = 2)
seu <- ScaleData(seu)
seu <- RunPCA(seu, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca",  dims = 1:2)

scConf8 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf8, gene.mapping = TRUE, shiny.prefix = "sc8", shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/", gex.assay = "ADT")

In [6]:
# Create Shiny app
makeShinyCodesMulti(
  shiny.title = "Ulcerative and Immune Checkpoint Therapy Colitis scRNAseq", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4", "sc5", "sc6", "sc7", "sc8"),
  shiny.headers = c( "PBMC RNA", "PBMC Protein", "Colon RNA", "Colon Protein","Epithelium RNA", "Epithelium Protein", "Stromal Immune RNA", "Stromal Immune Protein"), 
  shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/")

In [7]:
# Launch shiny app
system("R -e \"shiny::runApp('colon_pbmc_uc_scRNAseq_GSE189040')\"")

In [16]:
# Create merged shiny object
cd3 <- get(load(file = "objects/CD3.rda"))
epithelium <- get(load(file = "objects/epithelium.rda"))
pbmc <- get(load(file = "objects/PBMC.rda"))
stroma <- get(load(file = "objects/stroma.rda"))

In [17]:
cd3@meta.data$Cell_Category <- "CD3"
epithelium@meta.data$Cell_Category <- "Epithelium"
pbmc@meta.data$Cell_Category <- "PBMC"
stroma@meta.data$Cell_Category <- "Stroma_Immune"

In [18]:
obj <- merge(x = cd3, y = c(epithelium, pbmc, stroma))
save(obj, file = file.path("objects", "GSE189185.rda"))

Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


In [19]:
unique(obj@meta.data$Cell_Category)

[1] "CD3"           "Epithelium"    "PBMC"          "Stroma_Immune"

In [22]:
obj <- subset(obj, subset = Cell_Category == "CD3")
unique(obj@meta.data$Cell_Type)

[1] "Th1"              "Tfh"              "Tregs"            "GZMH+ Effector"  
 [5] "Th17 PD1-"        "Naive CD4+"       "CD103+ TRM"       "GZMK+ Effector"  
 [9] "Th17 PD1+"        "CD103- HAVCR2+"   "CD103+ HAVCR2+"   "CD103+ ZNF683+"  
[13] "γδ-rich 2"        "IFN Response"     "Cycling"          "Tph"             
[17] "γδ-rich 1"        "MAITs"            "FGFBP2+ Effector" "Naive CD8+"